## BBC News Classification

Completed as part of CU Boulder's Unsupervised Algorithms in Machine Learning course.

In [2]:
# Project dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math

In [8]:
# Data import
train = pd.read_csv("data/BBC News Train.csv")
test = pd.read_csv("data/BBC News Test.csv")

### Exploratory Data Analysis (EDA)

In [9]:

print(train.head(5))
print("\nTrain:", train.info())
print("\nTest:", test.info())

   ArticleId                                               Text  Category
0       1833  worldcom ex-boss launches defence lawyers defe...  business
1        154  german business confidence slides german busin...  business
2       1101  bbc poll indicates economic gloom citizens in ...  business
3       1976  lifestyle  governs mobile choice  faster  bett...      tech
4        917  enron bosses in $168m payout eighteen former e...  business
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ArticleId  1490 non-null   int64 
 1   Text       1490 non-null   object
 2   Category   1490 non-null   object
dtypes: int64(1), object(2)
memory usage: 35.0+ KB

Train: None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735 entries, 0 to 734
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Arti

In [11]:
# Check for any null/null-like values
null_like = [np.nan, None, [], {}, 'NaN', 'Null','NULL','None','NA','?','-', '.', '', ' ', '   ']

for df in [train, test]:
    for c in df.columns:
        string_null = np.array([x in null_like for x in df[c]])
        print(c, df[c].isnull().sum(), string_null.sum())

ArticleId 0 0
Text 0 0
Category 0 0
ArticleId 0 0
Text 0 0


In [ ]:
fig, a = plt.subplots(1,3) 
fig.set_figheight(12)
fig.set_figwidth(14)
fig.tight_layout()

for i, c in enumerate(cleaned.columns):
  data = cleaned[c]
  ax = a.flat[i]
  if data.dtype == 'bool':
    trues = np.count_nonzero(data)
    falses = np.size(data) - trues
    ax.bar(["True ({})".format(trues), "False ({})".format(falses)], [trues, falses])
  else:
    ax.hist(data, bins=50, log=True)
  ax.set_title(c)

for i in range(8,9):
  a.flat[i].set_visible(False)